# Stokes drift at ocean station Papa

This notebook computes Stokes drift from wave spectrum measured at the ocean station Papa. The wave buoy data is obtained from the [Coastal Data Information Program (CDIP)](http://cdip.ucsd.edu/themes/cdip?pb=1&u2=s:166:st:1&d2=p9) [data archive](http://thredds.cdip.ucsd.edu/thredds/catalog/cdip/archive/166p1/catalog.html).

Computing Stokes drift for one year at about 30 minutes interval takes about 10 min on a 2017 MacBook Pro.

In [ ]:
import sys
import os
import numpy as np
import xarray as xr
import pandas as pd
from scipy import special
sys.path.append(os.path.join(os.pardir, 'gotmtool'))
from gotmtool import *
from gotmtool.stokesdrift import stokes_drift_spec

### Load data

In [ ]:
datapath_wav = os.path.join(os.environ['HOME'], 'data', 'CDIP', 'Papa', '166p1_historic.nc') 

Select a subset of the data

In [ ]:
startdate = '2012-03-20'
enddate = '2013-03-22'
dss = xr.open_dataset(datapath_wav).sel(waveTime=slice(startdate, enddate))

In [ ]:
# time
time = dss.waveTime.values
# band center frequency
freq = dss.waveFrequency.values
# frequency bandwidth
dfreq = dss.waveBandwidth.values
# band energy density
spec = dss.waveEnergyDensity.values
# band mean direction that wave is coming from, in degree clockwise from the North
mdir = dss.waveMeanDirection.values
# angle in degree counterclockwise from the East
theta = -90.0-mdir
d2r = np.pi/180.0
xcmp = np.cos(theta*d2r)
ycmp = np.sin(theta*d2r)

### Vertical grid

In [ ]:
z1 = -np.linspace(0.5, 35.5, 36)
z2 = -np.linspace(40, 200, 17)
z = np.concatenate((z1,z2))
zi1 = -np.linspace(1, 36, 36)
zi2 = -np.linspace(45, 205, 17)
zi = np.concatenate((zi1,zi2))

### Compute Stokes drift

In [ ]:
%%time
ntime = time.size
nz = z.size
us0 = np.zeros([ntime])
vs0 = np.zeros([ntime])
us = np.zeros([ntime, nz])
vs = np.zeros([ntime, nz])
us_t = np.zeros([ntime, nz])
vs_t = np.zeros([ntime, nz])
for i in np.arange(ntime):
    us0[i], vs0[i] = stokes_drift_spec(np.zeros(1), spec[i,:], xcmp[i,:], ycmp[i,:], freq, dfreq)
    us[i,:], vs[i,:] = stokes_drift_spec(z, spec[i,:], xcmp[i,:], ycmp[i,:], freq, dfreq)

In [ ]:
da_time = xr.DataArray(
    time,
    dims=('time'),
    coords={'time': time},
    attrs={'long_name': 'UTC sample start time'},
    )
da_z = xr.DataArray(
    z,
    dims=('z'),
    coords={'z': z},
    attrs={'long_name': 'depth', 'units': 'm'},
    )
da_us0 = xr.DataArray(
    us0,
    dims=('time'),
    coords={'time': da_time},
    attrs={'long_name': 'Surface Stokes drift x-component', 'units': 'm/s'},
    )
da_vs0 = xr.DataArray(
    vs0,
    dims=('time'),
    coords={'time': da_time},
    attrs={'long_name': 'Surface Stokes drift y-component', 'units': 'm/s'},
    )
da_us = xr.DataArray(
    us,
    dims=('time', 'z'),
    coords={'z': da_z, 'time': da_time},
    attrs={'long_name': 'Stokes drift x-component', 'units': 'm/s'},
    )
da_vs = xr.DataArray(
    vs,
    dims=('time', 'z'),
    coords={'z': da_z, 'time': da_time},
    attrs={'long_name': 'Stokes drift y-component', 'units': 'm/s'},
    )
ds_out = da_us.to_dataset(name='us')
ds_out['vs'] = da_vs
ds_out['us0'] = da_us0
ds_out['vs0'] = da_vs0
ds_out.to_netcdf('ospapa_stokes_drift.nc')

### Convert time from `numpy.datetime64` to `datetime.datetime`

In [ ]:
dttime = [pd.Timestamp(time[i]).to_pydatetime() for i in np.arange(ntime)]

### Save Stokes drift to file

In [ ]:
dat_dump_ts(dttime, [us0, vs0], 'us_surface.dat')
dat_dump_pfl(dttime, z, [us, vs], 'us_prof.dat')